In [2]:
!pip install requests
!pip install bs4
!pip install lxml
!pip install pandas

In [1]:
# import libraries
import os
import pandas as pd
import lxml
import requests  
from bs4 import BeautifulSoup
from scraper import *

# scraping of the performance info

In [6]:
from scraper import *
from utils import *

list_url = ['https://www.vlr.gg/318937/karmine-corp-vs-team-heretics-champions-tour-2024-emea-stage-1-w5', 'https://www.vlr.gg/318930/team-liquid-vs-team-heretics-champions-tour-2024-emea-stage-1-w4']

test = general_data_scraper(list_url)

In [7]:
test

,Player Name,Agent Name,R,ACS,K,D,K/D +/–,+/–,ADR,HS%,...,FK/FD +/–,+/–,Team Name,Map Name,Map #,Stage,Series,winner,Id,rounds
0,MAGNUM,Cypher,1.68\n1.81\n1.40,287\n298\n266,18\n13\n5,\n\n6\n3\n3\n\n,8\n4\n4,+12\n+10\n+2,182\n191\n165,43%\n48%\n33%,...,2\n1\n1,+3\n+3\n0,Karmine Corp,Lotus,2,Regular Season,Week 5,Team Heretics,2,"1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1..."
1,tomaszy,Viper,1.51\n1.57\n1.39,237\n242\n229,17\n11\n6,\n\n7\n4\n3\n\n,5\n4\n1,+10\n+7\n+3,160\n149\n183,32%\n20%\n55%,...,0\n0\n0,+3\n+2\n+1,Karmine Corp,Lotus,2,Regular Season,Week 5,Team Heretics,2,"1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1..."
2,sh1n,Omen,1.30\n1.50\n0.89,175\n218\n90,12\n10\n2,\n\n7\n4\n3\n\n,7\n1\n6,+5\n+6\n-1,105\n144\n28,39%\n50%\n0%,...,0\n0\n0,+3\n+3\n0,Karmine Corp,Lotus,2,Regular Season,Week 5,Team Heretics,2,"1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1..."
3,marteen,Raze,1.23\n1.27\n1.16,227\n206\n271,14\n9\n5,\n\n10\n6\n4\n\n,6\n3\n3,+4\n+3\n+1,156\n158\n152,26%\n30%\n19%,...,2\n1\n1,+1\n0\n+1,Karmine Corp,Lotus,2,Regular Season,Week 5,Team Heretics,2,"1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1..."
4,N4RRATE,Fade,0.98\n1.13\n0.68,132\n123\n152,9\n6\n3,\n\n9\n5\n4\n\n,6\n4\n2,0\n+1\n-1,94\n91\n101,21%\n25%\n15%,...,0\n0\n0,0\n0\n0,Karmine Corp,Lotus,2,Regular Season,Week 5,Team Heretics,2,"1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1..."
5,Wo0t,Viper,0.91\n0.91\n0.92,168\n130\n188,10\n3\n7,\n\n12\n3\n9\n\n,3\n1\n2,-2\n0\n-2,118\n92\n132,26%\n20%\n28%,...,4\n1\n3,-4\n-1\n-3,Team Heretics,Lotus,2,Regular Season,Week 5,Team Heretics,3,"0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0..."
6,Boo,Omen,0.74\n0.93\n0.65,168\n247\n129,8\n4\n4,\n\n14\n5\n9\n\n,8\n3\n5,-6\n-1\n-5,109\n150\n89,26%\n50%\n16%,...,1\n0\n1,+2\n+1\n+1,Team Heretics,Lotus,2,Regular Season,Week 5,Team Heretics,3,"0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0..."
7,benjyfishy,Killjoy,0.68\n1.10\n0.47,125\n275\n51,8\n6\n2,\n\n13\n4\n9\n\n,1\n0\n1,-5\n+2\n-7,92\n192\n41,44%\n50%\n33%,...,1\n0\n1,-1\n0\n-1,Team Heretics,Lotus,2,Regular Season,Week 5,Team Heretics,3,"0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0..."
8,MiniBoo,Neon,0.42\n0.77\n0.24,119\n134\n113,7\n3\n4,\n\n16\n4\n12\n\n,5\n1\n4,-9\n-1\n-8,75\n80\n72,33%\n38%\n31%,...,3\n0\n3,-3\n0\n-3,Team Heretics,Lotus,2,Regular Season,Week 5,Team Heretics,3,"0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0..."
9,RieNs,Gekko,0.30\n0.13\n0.38,92\n40\n119,5\n1\n4,\n\n15\n5\n10\n\n,1\n0\n1,-10\n-4\n-6,62\n23\n81,31%\n33%\n31%,...,5\n3\n2,-4\n-2\n-2,Team Heretics,Lotus,2,Regular Season,Week 5,Team Heretics,3,"0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0..."


In [1]:
from scraper import *

main("https://www.vlr.gg/event/matches/1924/champions-tour-2024-pacific-kickoff/?series_id=all")

Get all the matches...
Fetching General Data...
Fetching Performance Data...
Fetching Economy Data...
Fetching Picks and Bans Data...
Saving the data...
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\general_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\performance_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\economy_data_champions-tour-2024-pacific-kickoff.csv
DataFrame saved as CSV file: champions-tour-2024-pacific-kickoff_data\pick_ban_data_champions-tour-2024-pacific-kickoff.csv


0

In [6]:
result.head()

,Stage,Series,Team Name,Bans,Picks,Decider
0,Group Stage,Decider (C),M8,"[Bind, Ascent]",[Breeze],[Sunset]
1,Group Stage,Decider (C),VIT,"[Icebox, Split]",[Lotus],[Sunset]
2,Play-Ins,Group B,GTZ,"[Lotus, Split, Haven]",[],[Pearl]
3,Play-Ins,Group B,MOUZ,"[Fracture, Ascent, Bind]",[],[Pearl]
